In [ ]:
# Using earthaccess access pattern for direct access "streaming" of cloud-hosted 
# MEaSUREs Phase-Based Antarctica Ice Velocity Map, Version 1
# 
# Code  written to run on CryoCloud cloud-computing JupyterHub
# Learn more: https://cryointhecloud.com/
# 
# Written 2023-11-15 by Wilson Sauthoff (wsauthoff.github.io)

In [ ]:
# Import libraries
import earthaccess
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import xarray as xr

In [ ]:
# Log into NASA Earthdata to search for datasets
earthaccess.login()

In [ ]:
# Find cloud-hosted MEaSUREs Phase-Based Antarctica Ice Velocity Map, Version 1
# DOI from https://nsidc.org/data/NSIDC-0754/versions/1
results = earthaccess.search_data(
    doi='10.5067/FPSU0V1MWUB6',
    cloud_hosted=True,
    bounding_box=(1, -89, -1, -89)  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
)

In [ ]:
# Open data granules as s3 files to stream
files = earthaccess.open(results)
files

In [ ]:
# Print file name to ensure expected dataset
print(files[0])

In [ ]:
# Open each file, which are quadrants in polar stereographic coordinations around the Geographic South Pole
bedmachine = xr.open_dataset(files[0])
bedmachine

In [ ]:
# Specify the variables to keep
variables_to_keep = ['x', 'y', 'surface', 'thickness', 'bed']

variables_to_drop = [var for var in bedmachine.variables if var not in variables_to_keep]

# Drop variables to reduce memory consumption
bedmachine = bedmachine.drop_vars(variables_to_drop)
bedmachine

In [ ]:
# Define bounding box of interest in polar stereographic coordinates
# (here Slessor Glacier, East Antarctica)
xmin=-600000
ymin=870000
xmax=-180000
ymax=1290000

# Subset BedMachine data within bounding box, assign to variable
bedmachine_sub = bedmachine.where((bedmachine.x > xmin) & (bedmachine.x < xmax) & (bedmachine.y > ymin) & (bedmachine.y < ymax), drop=True)

In [ ]:
# Visualize bed elevation 
fig, ax = plt.subplots(figsize=(5,5))
m = ax.imshow(bedmachine['bed'],
    extent=[bedmachine.x.min(), bedmachine.x.max(), bedmachine.y.min(), bedmachine.y.max()],
    cmap='gist_earth')
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
#TODO plotting along pixel values, but using axes limits plots nothing; remendy plotting issue
ax.set_xlim(bedmachine.x.min(), bedmachine.x.max())
ax.set_ylim(bedmachine.y.min(), bedmachine.y.max())
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.2)
cbar = fig.colorbar(mappable, cax=cax)
cbar.set_label('bed elevation [m ASL]')
plt.show()